In [ ]:
import Pkg
Pkg.activate("../../../")

In [1]:
from laplace import Laplace

In [2]:
import pandas as pd
import torch

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(43)

# Load data from CSV file using pandas
df = pd.read_csv('data1.csv')

# Split the dataframe into x and y tensors
x = torch.from_numpy(df[['x1', 'x2']].to_numpy()).to(torch.float32)
y = torch.from_numpy(df['y'].to_numpy(dtype=int))

In [3]:
X = torch.tensor(x.T).float().T

# Convert y to a tensor of indices and one-hot encode it
y_unique = torch.unique(torch.tensor(y))
y_indices = torch.searchsorted(y_unique, torch.tensor(y))
y_train = nn.functional.one_hot(y_indices, num_classes=len(y_unique)).float()


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4736\1740726160.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(x.T).float().T
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4736\1740726160.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_unique = torch.unique(torch.tensor(y))
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4736\1740726160.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_indices = torch.searchsorted(y_unique, torch.tensor(y))


In [4]:
data = list(zip(x, y_train))
n_hidden = 3
D = X.shape[1]
out_dim = y_train.shape[1]
model = nn.Sequential(
    nn.Linear(D, n_hidden),
    nn.Sigmoid(),
    nn.Linear(n_hidden, out_dim)
)

In [5]:
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters())
epochs = 200
avg_loss = lambda data: torch.mean(torch.stack([loss_fn(model(x), y) for (x, y) in data]))
show_every = epochs // 10

In [6]:
for epoch in range(1, epochs+1):
    for (x, y) in data:
        opt.zero_grad()
        loss = loss_fn(model(x), y)
        loss.backward()
        opt.step()
    if epoch % show_every == 0:
        print("Epoch ", epoch)
        print("Avg Loss: ", avg_loss(data).item())

Epoch  20
Avg Loss:  0.661486029624939
Epoch  40
Avg Loss:  0.3299190402030945
Epoch  60
Avg Loss:  0.17765741050243378
Epoch  80
Avg Loss:  0.09842698276042938
Epoch  100
Avg Loss:  0.05515485629439354
Epoch  120
Avg Loss:  0.031066937372088432
Epoch  140
Avg Loss:  0.017541011795401573
Epoch  160
Avg Loss:  0.009913784451782703
Epoch  180
Avg Loss:  0.00560393463820219
Epoch  200
Avg Loss:  0.0031665244605392218


In [7]:
from laplace.curvature import AsdlEF
la = Laplace(model, 'classification',
             subset_of_weights='all',
             hessian_structure='full', backend=AsdlEF)

In [8]:
la.fit(DataLoader(TensorDataset(X, y_train), batch_size=1))

In [9]:
la.optimize_prior_precision()

In [10]:
print(la.backend)

In [11]:
probit_predictions = la(X, link_approx='probit')

In [12]:
predictions_probit_df = pd.DataFrame(probit_predictions.numpy(), columns=['class1', 'class2', 'class3', 'class4'])

In [13]:
predictions_probit_df.to_csv('predictions1-Python.csv', index=False)

In [14]:
print(probit_predictions)

tensor([[0.5963, 0.0597, 0.2262, 0.1178],
        [0.6057, 0.0584, 0.2224, 0.1136],
        [0.6025, 0.0590, 0.2231, 0.1154],
        [0.6056, 0.0584, 0.2223, 0.1136],
        [0.6053, 0.0585, 0.2223, 0.1139],
        [0.6057, 0.0584, 0.2224, 0.1136],
        [0.6055, 0.0584, 0.2223, 0.1137],
        [0.6057, 0.0584, 0.2224, 0.1136],
        [0.6055, 0.0584, 0.2223, 0.1137],
        [0.6044, 0.0586, 0.2226, 0.1144],
        [0.6047, 0.0586, 0.2224, 0.1143],
        [0.4046, 0.0782, 0.2961, 0.2211],
        [0.6046, 0.0586, 0.2225, 0.1143],
        [0.6056, 0.0584, 0.2223, 0.1137],
        [0.6057, 0.0584, 0.2224, 0.1136],
        [0.6056, 0.0584, 0.2223, 0.1136],
        [0.6056, 0.0584, 0.2223, 0.1136],
        [0.6044, 0.0588, 0.2223, 0.1146],
        [0.6057, 0.0584, 0.2224, 0.1136],
        [0.6056, 0.0584, 0.2224, 0.1136],
        [0.6057, 0.0584, 0.2224, 0.1136],
        [0.5870, 0.0614, 0.2284, 0.1232],
        [0.5278, 0.0676, 0.2563, 0.1482],
        [0.6057, 0.0584, 0.2224, 0

In [15]:
print(torch.diag(la.prior_precision_diag))

tensor([[0.0244, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0244, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0244, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0244, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0244, 0.0000, 0.0000, 0.0000, 0.0000,
       

In [16]:
torch.set_printoptions(sci_mode=False)
print(torch.softmax(model(X), dim=1))

tensor([[    0.9966,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9966,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9966,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9967,     0.0000,     0.0030,     0.0003],
        [    0.9966,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0.9965,     0.0000,     0.0034,     0.0001],
        [    0

In [17]:
f_mu, f_var = la._glm_predictive_distribution(X)

In [18]:
print(f_mu)

tensor([[  4.4230,  -8.8740,  -1.2699,  -5.0352],
        [  4.4861,  -8.8812,  -1.1993,  -5.0710],
        [  4.4436,  -8.8760,  -1.2467,  -5.0467],
        [  4.4845,  -8.8810,  -1.2010,  -5.0701],
        [  4.4778,  -8.8802,  -1.2087,  -5.0662],
        [  4.4863,  -8.8812,  -1.1991,  -5.0711],
        [  4.4810,  -8.8800,  -1.2033,  -5.0695],
        [  4.4862,  -8.8812,  -1.1992,  -5.0710],
        [  4.4823,  -8.8804,  -1.2026,  -5.0696],
        [  4.4666,  -8.8789,  -1.2212,  -5.0599],
        [  4.4677,  -8.8790,  -1.2199,  -5.0605],
        [  3.5402,  -8.7660,  -2.2549,  -4.5300],
        [  4.4663,  -8.8789,  -1.2215,  -5.0597],
        [  4.4844,  -8.8810,  -1.2012,  -5.0700],
        [  4.4861,  -8.8812,  -1.1993,  -5.0710],
        [  4.4855,  -8.8811,  -1.2000,  -5.0706],
        [  4.4857,  -8.8811,  -1.1998,  -5.0707],
        [  4.4506,  -8.8739,  -1.2341,  -5.0530],
        [  4.4862,  -8.8812,  -1.1992,  -5.0710],
        [  4.4852,  -8.8809,  -1.1999,  -5.0709],


In [19]:
print(f_var.diagonal(dim1=1, dim2=2))

tensor([[ 85.0710,  81.2067,  85.9939,  83.1064],
        [ 79.5333,  81.1535,  79.0446,  81.3597],
        [ 80.9911,  81.1568,  80.8741,  81.8108],
        [ 79.5348,  81.1529,  79.0466,  81.3597],
        [ 79.6317,  81.1522,  79.1680,  81.3888],
        [ 79.5332,  81.1536,  79.0445,  81.3597],
        [ 79.5396,  81.1510,  79.0523,  81.3599],
        [ 79.5332,  81.1536,  79.0445,  81.3597],
        [ 79.5376,  81.1516,  79.0500,  81.3597],
        [ 80.0510,  81.1541,  79.6939,  81.5193],
        [ 79.8721,  81.1516,  79.4696,  81.4622],
        [809.4590,  91.1123, 995.0504, 314.0158],
        [ 79.9501,  81.1523,  79.5675,  81.4868],
        [ 79.5390,  81.1531,  79.0518,  81.3611],
        [ 79.5332,  81.1535,  79.0446,  81.3597],
        [ 79.5340,  81.1534,  79.0455,  81.3598],
        [ 79.5336,  81.1534,  79.0450,  81.3597],
        [ 79.7726,  81.1247,  79.3484,  81.4129],
        [ 79.5332,  81.1536,  79.0445,  81.3597],
        [ 79.5335,  81.1532,  79.0449,  81.3596],


In [20]:
torch.set_printoptions(sci_mode=False)
print(la.posterior_covariance)

tensor([[    41.0045,     -0.0002,     -0.0000,      0.0000,      0.0000,
              0.0000,     -0.0000,     -0.0000,      0.0000,      0.0001,
             -0.0010,     -0.0001,     -0.0002,     -0.0000,      0.0000,
             -0.0001,      0.0011,      0.0001,      0.0002,     -0.0000,
             -0.0000,     -0.0010,     -0.0002,      0.0010,      0.0002],
        [    -0.0002,     41.0039,      0.0000,      0.0001,      0.0000,
              0.0000,     -0.0001,      0.0000,      0.0000,      0.0004,
              0.0000,      0.0004,      0.0003,      0.0002,      0.0003,
             -0.0004,      0.0000,     -0.0006,     -0.0003,     -0.0002,
             -0.0001,      0.0000,      0.0005,     -0.0000,     -0.0004],
        [    -0.0000,      0.0000,     41.0045,     -0.0000,      0.0000,
              0.0000,     -0.0000,     -0.0000,      0.0000,     -0.0000,
             -0.0003,      0.0001,     -0.0001,      0.0000,     -0.0000,
              0.0000,      0.0002,  

In [21]:
print(la.H)

tensor([[     0.0000,      0.0000,      0.0000,     -0.0000,     -0.0000,
             -0.0000,      0.0000,      0.0000,     -0.0000,     -0.0000,
              0.0000,      0.0000,      0.0000,      0.0000,     -0.0000,
              0.0000,     -0.0000,     -0.0000,     -0.0000,      0.0000,
              0.0000,      0.0000,      0.0000,     -0.0000,     -0.0000],
        [     0.0000,      0.0000,     -0.0000,     -0.0000,     -0.0000,
             -0.0000,      0.0000,     -0.0000,     -0.0000,     -0.0000,
             -0.0000,     -0.0000,     -0.0000,     -0.0000,     -0.0000,
              0.0000,      0.0000,      0.0000,      0.0000,      0.0000,
              0.0000,     -0.0000,     -0.0000,      0.0000,      0.0000],
        [     0.0000,     -0.0000,      0.0000,      0.0000,     -0.0000,
             -0.0000,      0.0000,      0.0000,     -0.0000,      0.0000,
              0.0000,     -0.0000,      0.0000,     -0.0000,      0.0000,
             -0.0000,     -0.0000,  

In [22]:
print(torch.diag(la.prior_precision_diag))

tensor([[0.0244, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0244, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0244, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0244, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0244, 0.0000, 0.0000, 0.0000, 0.0000,
       

In [23]:
torch.set_printoptions(sci_mode=False)
Js, f_mu = la.backend.jacobians(X)
Js = Js.permute(2, 0, 1).contiguous().view(25, -1)
for i in range(6):
    print(Js[i, :3])

print()
for i in range(6, 9):
    print(Js[i, :3])
print()
for i in range(9, 21):
    print(Js[i, :3])
print()
for i in range(21, 25):
    print(Js[i, :3])

tensor([-0.1412,  0.0156, -0.1582])
tensor([-0.3334,  0.0370, -0.3735])
tensor([     0.0002,     -0.0005,      0.0000])
tensor([     0.0005,     -0.0013,      0.0001])
tensor([    -0.0000,      0.0000,      0.0000])
tensor([    -0.0000,      0.0000,      0.0000])

tensor([-0.0625,  0.0069, -0.0700])
tensor([     0.0001,     -0.0002,      0.0000])
tensor([    -0.0000,      0.0000,      0.0000])

tensor([0.0118, 0.0000, 0.0000])
tensor([1.0000, 0.0000, 0.0000])
tensor([    0.0000,     0.0000,     0.0000])
tensor([0.0000, 0.0118, 0.0000])
tensor([0.0000, 1.0000, 0.0000])
tensor([    0.0000,     0.0000,     0.0000])
tensor([0.0000, 0.0000, 0.0118])
tensor([0.0000, 0.0000, 1.0000])
tensor([    0.0000,     0.0000,     0.0000])
tensor([0., 0., 0.])
tensor([0., 0., 0.])
tensor([0., 0., 0.])

tensor([1., 0., 0.])
tensor([0., 1., 0.])
tensor([0., 0., 1.])
tensor([0., 0., 0.])


In [24]:
from asdfghjkl.hessian import hessian_for_loss


In [25]:
train_loader = DataLoader(TensorDataset(X, y_train), batch_size=1)
N = len(train_loader.dataset)

for x, y in train_loader:
    print(x)
    print(y)
    loss_batch, H_batch = la._curv_closure(x, y, N)
    print(H_batch)
    print()


tensor([[2.2603, 5.3383]])
tensor([[1., 0., 0., 0.]])
tensor([[     0.0000,      0.0000,      0.0000,      0.0000,     -0.0000,
             -0.0000,      0.0000,      0.0000,     -0.0000,      0.0000,
              0.0000,      0.0000,     -0.0000,     -0.0000,     -0.0000,
             -0.0000,     -0.0000,     -0.0000,     -0.0000,     -0.0000,
             -0.0000,      0.0000,     -0.0000,     -0.0000,     -0.0000],
        [     0.0000,      0.0000,      0.0000,      0.0000,     -0.0000,
             -0.0000,      0.0000,      0.0000,     -0.0000,      0.0000,
              0.0000,      0.0000,     -0.0000,     -0.0000,     -0.0000,
             -0.0000,     -0.0000,     -0.0000,     -0.0000,     -0.0000,
             -0.0000,      0.0000,     -0.0000,     -0.0000,     -0.0000],
        [     0.0000,      0.0000,      0.0000,      0.0000,     -0.0000,
             -0.0000,      0.0000,      0.0000,     -0.0000,      0.0000,
              0.0000,      0.0000,     -0.0000,     -0.0

In [26]:
import math
row = [ 1.5734e-09,  3.7159e-09,  2.7287e-11,  6.4444e-11, -4.9952e-17,
         -1.1797e-16,  6.9608e-10,  1.2072e-11, -2.2099e-17,  1.6122e-09,
          1.3635e-07,  1.8970e-18, -7.8499e-13, -6.6390e-11, -9.2367e-22,
         -1.5749e-09, -1.3320e-07, -1.8532e-18, -3.6475e-11, -3.0849e-09,
         -4.2919e-20,  1.3635e-07, -6.6393e-11, -1.3320e-07, -3.0850e-09]

a = math.sqrt(row[0])
g = []
for elem in row:
    g.append(elem / a)
print(g)

[3.966610643862087e-05, 9.367947433282783e-05, 6.879172787534306e-07, 1.6246616012015274e-06, -1.2593119034079e-12, -2.974075617493393e-12, 1.7548483138296187e-05, 3.0434043277426664e-07, -5.571255155631642e-13, 4.06442715141379e-05, 0.0034374435063594484, 4.78242048519536e-14, -1.978994336675543e-08, -1.6737211176179228e-06, -2.3286127198526084e-17, -3.970392209875684e-05, -0.0033580306200739164, -4.671998757598335e-14, -9.195508023062772e-07, -7.777168663563082e-05, -1.0820068782503937e-15, 0.0034374435063594484, -1.673796748938195e-06, -0.0033580306200739164, -7.777420767963989e-05]


In [27]:
Js, f_mu = la.backend.jacobians(X)
ps = torch.softmax(f_mu, dim=-1)

In [28]:
print(torch.einsum('mk,mc->mck', ps, ps).shape)

torch.Size([100, 4, 4])


In [29]:
print(Js.shape)

torch.Size([100, 4, 25])


In [30]:
print(X[0])

tensor([2.2603, 5.3383])


In [31]:
print(la.backend)